In [73]:
import pandas as pd
import seaborn as sns
import requests
import numpy as np
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize,SyllableTokenizer

In [2]:
df=pd.read_excel('Input.xlsx')

In [3]:
df

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [4]:
# Extracting the Title..

In [5]:
title=[]
for url in df.URL:
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        useful_data = soup.find('h1', class_='entry-title').text
        title.append(useful_data)
    except AttributeError:
        title.append(url.split("/")[3].replace('-'," ").title())
        

In [6]:
len(title)

100

In [7]:
df['title']=title

In [8]:
# Extracting the article text from web

In [9]:
text=[]
for i in df.URL:
    try:
        r=requests.get(i)
        soup=BeautifulSoup(r.text,'lxml')
        useful_data=soup.find('div',class_='td-post-content tagdiv-type').text
        text.append(useful_data)
    except AttributeError:
        r=requests.get(i)
        soup=BeautifulSoup(r.text,'lxml')
        useful_data=soup.find('div',class_='tdb-block-inner td-fix-index').text
        text.append(useful_data)

In [10]:
len(text)

100

In [11]:
df['article_text']=text

In [12]:
df

,URL_ID,URL,title,article_text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,\nWe have seen a huge development and dependen...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"\nThroughout history, from the industrial revo..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",\nIntroduction\nIn the span of just a few deca...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"\nThe way we live, work, and communicate has u..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,\nThe year 2040 is poised to witness a continu...
...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,"\nEpidemics, in general, have both direct and ..."
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,\nCOVID 19 has bought the world to its knees. ...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,Contribution of handicrafts (Visual Arts & Lit...,\nHandicrafts is an art of making crafts by ha...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,How Covid 19 Is Impacting Payment Preferences,


In [22]:
df=df[df.article_text!=''].reset_index(drop=True)

# Working On Stopwords

In [24]:
# Stopwords removal

# Importing stopword files
st1=pd.read_table('StopWords_Auditor.txt',encoding='utf-8',header=None)
st2=pd.read_table('StopWords_Currencies.txt',encoding='latin-1',header=None)
st3=pd.read_table('StopWords_Generic.txt',encoding='utf-8',header=None)
st4=pd.read_table('StopWords_Geographic.txt',encoding='utf-8',header=None)
st5=pd.read_table('StopWords_DatesandNumbers.txt',encoding='utf-8',header=None)
st6=pd.read_table('StopWords_GenericLong.txt',encoding='utf-8',header=None)
st7=pd.read_table('StopWords_Names.txt',encoding='utf-8',header=None)


In [25]:
# creating a master stopwords list

# combining all stopwords imported files into one list
s_lists=[]
for i in range(1,8):
    st_list_name = 'st{}'.format(i)
    s_list = []
    for x in globals()[st_list_name][0]:
        s_list.append(x)
    s_lists.append(s_list)


In [26]:
# Master stopwords list
mstr=[]
for i in range(0,7):
    lst=s_lists[i]
    for x in lst:
        if type(x)==str:
            mstr.append(x.split('|')[0].rstrip())
        else:
            continue

In [27]:
'SMITH  | Surnames from 1990 census > .002%.  www.census.gov.genealogy/names/dist.all.last'.split('|')[0].rstrip()

'SMITH'

In [28]:
#Converting the Uppercase into lower case
stop_words=[]
for i in mstr:
    stop_words.append(i.lower())

In [29]:
len(stop_words)

14105

In [127]:
stop_words.append(',')
stop_words.append('!')
stop_words.append('.')
stop_words.append('?')

In [128]:
# Stopwords removal from the article
cleaned_txt=[]
for i in df.article_text:
    tokens=word_tokenize(i.lower())
    clean_txt=[]
    for x in tokens:
        if x not in stop_words:
            clean_txt.append(x)
    txt=' '.join(clean_txt)
    cleaned_txt.append(txt)

In [129]:
# Creating a new column which consists the clean text
df['cleaned_txt']=cleaned_txt

In [130]:
df

,URL_ID,URL,title,article_text,cleaned_txt,tokenized_text,pos_count,neg_count,polarity_score,total_words,subjectivity_score,no_of_sent,sent_avg_len,complex_word_count,per_complex_words,fog_index
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,\nWe have seen a huge development and dependen...,huge development dependence people technology ...,"[huge, development, dependence, people, techno...",26,6,0.625000,597,0.053601,64,9.328125,180,30.15,0.309391
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"\nThroughout history, from the industrial revo...",history industrial revolution 18th century dev...,"[history, industrial, revolution, 18th, centur...",53,31,0.261905,794,0.105793,80,9.925000,371,46.73,0.212390
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",\nIntroduction\nIn the span of just a few deca...,introduction span decades internet undergone a...,"[introduction, span, decades, internet, underg...",36,23,0.220339,677,0.087149,57,11.877193,376,55.54,0.213849
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"\nThe way we live, work, and communicate has u...",live work communicate unquestionably changed r...,"[live, work, communicate, unquestionably, chan...",37,74,-0.333333,667,0.166417,47,14.191489,344,51.57,0.275189
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,\nThe year 2040 is poised to witness a continu...,2040 poised witness continued revolution world...,"[2040, poised, witness, continued, revolution,...",19,8,0.407407,382,0.070681,39,9.794872,169,44.24,0.221403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,Gaming Disorder and Effects of Gaming on Health.,\nPerhaps the virtual illusion has become toda...,virtual illusion today ’ “ reality ” inquisiti...,"[virtual, illusion, today, ’, “, reality, ”, ....",28,45,-0.232877,596,0.122483,62,9.612903,210,35.23,0.272861
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,What is the repercussion of the environment du...,\nWhat is COVID 19 pandemic?\nOn 31st December...,covid 19 pandemic 31st 2019 coronavirus identi...,"[covid, 19, pandemic, ?, 31st, 2019, coronavir...",6,24,-0.600000,369,0.081301,35,10.542857,128,34.69,0.303916
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,"\nEpidemics, in general, have both direct and ...",epidemics general direct indirect costs direct...,"[epidemics, general, direct, indirect, costs, ...",26,54,-0.350000,611,0.130933,48,12.729167,276,45.17,0.281806
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,\nCOVID 19 has bought the world to its knees. ...,covid 19 bought world knees businesses shut tr...,"[covid, 19, bought, world, knees, ., businesse...",22,35,-0.228070,467,0.122056,38,12.289474,178,38.12,0.322389


In [131]:
len(df.article_text.iloc[1])

9698

In [132]:
len(df.cleaned_txt.iloc[1])

6081

In [133]:
# Importing Master Dict
neg=pd.read_table('negative-words.txt',encoding='latin1',header=None)
pos=pd.read_table('positive-words.txt',encoding='latin1',header=None)

In [134]:
positive=[]
for i in pos[0]:
    positive.append(i)
    
negative=[]
for i in neg[0]:
    negative.append(i)

In [135]:
# Stopwords removal from the negative words dict
neg_cleaned=[]
for i in neg[0]:
    tokens=word_tokenize(i.lower())
    clean_txt=[]
    for x in tokens:
        if x not in stop_words:
            clean_txt.append(x)
    txt=' '.join(clean_txt)
    neg_cleaned.append(txt)
    
# Stopwords removal from the positive words dict
pos_cleaned=[]
for i in pos[0]:
    tokens=word_tokenize(i.lower())
    clean_txt=[]
    for x in tokens:
        if x not in stop_words:
            clean_txt.append(x)
    txt=' '.join(clean_txt)
    pos_cleaned.append(txt)

In [136]:
# After stopwords removal from neg dict
neg_cleaned

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [137]:
# After stopwords removal from positive dict
pos_cleaned

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [138]:
tokenized_articles = []

for text in df.cleaned_txt:
    tokens = word_tokenize(text.lower())
    tokenized_articles.append(tokens)

df['tokenized_text'] = tokenized_articles


In [139]:
#finding total length
total_wrds= []
for i in df.tokenized_text:
    total_wrds.append(len(i))
df['total_words']=total_wrds

In [140]:
# Finding Positive, negative, polarity score, Subjectivity Score

num_pos = df['tokenized_text'].map(lambda x: len([i for i in x if i in pos_cleaned]))
df['pos_count'] = num_pos

In [141]:
# Finding Positive, negative, polarity score, Subjectivity Score

num_neg = df['tokenized_text'].map(lambda x: len([i for i in x if i in neg_cleaned]))
df['neg_count'] = num_neg

In [142]:
#finding polarity score
def polarity(pos,neg):
    polarity_sc=(pos-neg)/((pos+neg)+0.000001)
    return polarity_sc

polarity_score=[]
for i,j in zip(df.pos_count,df.neg_count):
    polarity_score.append(polarity(i,j))

In [143]:
polarity(26,6)

0.6249999804687506

In [144]:
df['polarity_score']=polarity_score

In [145]:
# Subjectivity Score

#Total Words after cleaning
#df['total_words']=df.tokenized_text.apply(len)

def sub_sc(pos,neg,total_words):
    subje_sc=(pos+neg)/((total_words)+0.000001)
    return subje_sc


subjec_score=[]
for pos,neg,total_words in zip(df['pos_count'], df['neg_count'], df['total_words']):
    subjec_score.append(sub_sc(pos,neg,total_words))

In [146]:
df['subjectivity_score']=subjec_score

In [147]:
len(df.tokenized_text.iloc[0])

534

In [148]:
df

,URL_ID,URL,title,article_text,cleaned_txt,tokenized_text,pos_count,neg_count,polarity_score,total_words,subjectivity_score,no_of_sent,sent_avg_len,complex_word_count,per_complex_words,fog_index
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,\nWe have seen a huge development and dependen...,huge development dependence people technology ...,"[huge, development, dependence, people, techno...",26,6,0.625000,534,0.059925,64,9.328125,180,30.15,0.309391
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"\nThroughout history, from the industrial revo...",history industrial revolution 18th century dev...,"[history, industrial, revolution, 18th, centur...",53,31,0.261905,715,0.117483,80,9.925000,371,46.73,0.212390
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",\nIntroduction\nIn the span of just a few deca...,introduction span decades internet undergone a...,"[introduction, span, decades, internet, underg...",36,23,0.220339,621,0.095008,57,11.877193,376,55.54,0.213849
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"\nThe way we live, work, and communicate has u...",live work communicate unquestionably changed r...,"[live, work, communicate, unquestionably, chan...",37,74,-0.333333,621,0.178744,47,14.191489,344,51.57,0.275189
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,\nThe year 2040 is poised to witness a continu...,2040 poised witness continued revolution world...,"[2040, poised, witness, continued, revolution,...",19,8,0.407407,344,0.078488,39,9.794872,169,44.24,0.221403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,Gaming Disorder and Effects of Gaming on Health.,\nPerhaps the virtual illusion has become toda...,virtual illusion today ’ “ reality ” inquisiti...,"[virtual, illusion, today, ’, “, reality, ”, i...",28,45,-0.232877,535,0.136449,62,9.612903,210,35.23,0.272861
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,What is the repercussion of the environment du...,\nWhat is COVID 19 pandemic?\nOn 31st December...,covid 19 pandemic 31st 2019 coronavirus identi...,"[covid, 19, pandemic, 31st, 2019, coronavirus,...",6,24,-0.600000,335,0.089552,35,10.542857,128,34.69,0.303916
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,"\nEpidemics, in general, have both direct and ...",epidemics general direct indirect costs direct...,"[epidemics, general, direct, indirect, costs, ...",26,54,-0.350000,564,0.141844,48,12.729167,276,45.17,0.281806
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,\nCOVID 19 has bought the world to its knees. ...,covid 19 bought world knees businesses shut tr...,"[covid, 19, bought, world, knees, businesses, ...",22,35,-0.228070,430,0.132558,38,12.289474,178,38.12,0.322389


In [149]:
# Analysis of redability

# No of sentense available in cleaned text
df['no_of_sent']=df.cleaned_txt.apply(lambda x: len(sent_tokenize(x)))

In [150]:
# Finding Average sentense length

def avg_sent_len(word,sent):
    sent=word/sent
    return sent

avg_len = []
for words, sents in zip(df['total_words'], df['no_of_sent']):
    avg_len.append(avg_sent_len(words, sents))

In [151]:
df['sent_avg_len']=avg_len

In [152]:
df

,URL_ID,URL,title,article_text,cleaned_txt,tokenized_text,pos_count,neg_count,polarity_score,total_words,subjectivity_score,no_of_sent,sent_avg_len,complex_word_count,per_complex_words,fog_index
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,\nWe have seen a huge development and dependen...,huge development dependence people technology ...,"[huge, development, dependence, people, techno...",26,6,0.625000,534,0.059925,1,534.0,180,30.15,0.309391
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"\nThroughout history, from the industrial revo...",history industrial revolution 18th century dev...,"[history, industrial, revolution, 18th, centur...",53,31,0.261905,715,0.117483,1,715.0,371,46.73,0.212390
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",\nIntroduction\nIn the span of just a few deca...,introduction span decades internet undergone a...,"[introduction, span, decades, internet, underg...",36,23,0.220339,621,0.095008,1,621.0,376,55.54,0.213849
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"\nThe way we live, work, and communicate has u...",live work communicate unquestionably changed r...,"[live, work, communicate, unquestionably, chan...",37,74,-0.333333,621,0.178744,1,621.0,344,51.57,0.275189
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,\nThe year 2040 is poised to witness a continu...,2040 poised witness continued revolution world...,"[2040, poised, witness, continued, revolution,...",19,8,0.407407,344,0.078488,1,344.0,169,44.24,0.221403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,Gaming Disorder and Effects of Gaming on Health.,\nPerhaps the virtual illusion has become toda...,virtual illusion today ’ “ reality ” inquisiti...,"[virtual, illusion, today, ’, “, reality, ”, i...",28,45,-0.232877,535,0.136449,1,535.0,210,35.23,0.272861
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,What is the repercussion of the environment du...,\nWhat is COVID 19 pandemic?\nOn 31st December...,covid 19 pandemic 31st 2019 coronavirus identi...,"[covid, 19, pandemic, 31st, 2019, coronavirus,...",6,24,-0.600000,335,0.089552,1,335.0,128,34.69,0.303916
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,"\nEpidemics, in general, have both direct and ...",epidemics general direct indirect costs direct...,"[epidemics, general, direct, indirect, costs, ...",26,54,-0.350000,564,0.141844,1,564.0,276,45.17,0.281806
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,\nCOVID 19 has bought the world to its knees. ...,covid 19 bought world knees businesses shut tr...,"[covid, 19, bought, world, knees, businesses, ...",22,35,-0.228070,430,0.132558,1,430.0,178,38.12,0.322389


In [153]:
st=SyllableTokenizer()
# Count of complex words ###
def count_complex_words(text):
    complex_words_count = 0
    for word in text.split():
        if len(st.tokenize(word)) > 2:
            complex_words_count += 1
    return complex_words_count

complex_word_count=[]
for i in range(0,89):
    text_complex_cnt=[]
    for x in df.tokenized_text.iloc[i]:
        text_complex_cnt.append(count_complex_words(x))
    complex_word_count.append(sum(text_complex_cnt))
    
df['complex_word_count']=complex_word_count

C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: '–'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: '’'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: '“'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: '”'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: '‘'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\nltk\tokenize\sonority_sequencing.py:

In [154]:
# Percentage of  Complex words
def percentage(complex_word,word_len):
    return complex_word/word_len

per=[]
for i,j in zip(df['complex_word_count'],df['total_words']):
    per.append(round(percentage(i,j)*100,2))
    
df['per_complex_words']=per

In [155]:
# Calculating the fog Index

#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

def fog_index(avg_sent_len,per_comp_word):
    return avg_sent_len/per_comp_word

fog_index_=[]
for i,j in zip(df['sent_avg_len'],df['per_complex_words']):
    fog_index_.append(fog_index(i,j))
    
df['fog_index']=fog_index_

In [157]:
# Final Data
df

,URL_ID,URL,title,article_text,cleaned_txt,tokenized_text,pos_count,neg_count,polarity_score,total_words,subjectivity_score,no_of_sent,sent_avg_len,complex_word_count,per_complex_words,fog_index
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,\nWe have seen a huge development and dependen...,huge development dependence people technology ...,"[huge, development, dependence, people, techno...",26,6,0.625000,534,0.059925,1,534.0,180,33.71,15.840997
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"\nThroughout history, from the industrial revo...",history industrial revolution 18th century dev...,"[history, industrial, revolution, 18th, centur...",53,31,0.261905,715,0.117483,1,715.0,371,51.89,13.779148
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",\nIntroduction\nIn the span of just a few deca...,introduction span decades internet undergone a...,"[introduction, span, decades, internet, underg...",36,23,0.220339,621,0.095008,1,621.0,376,60.55,10.255987
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"\nThe way we live, work, and communicate has u...",live work communicate unquestionably changed r...,"[live, work, communicate, unquestionably, chan...",37,74,-0.333333,621,0.178744,1,621.0,344,55.39,11.211410
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,\nThe year 2040 is poised to witness a continu...,2040 poised witness continued revolution world...,"[2040, poised, witness, continued, revolution,...",19,8,0.407407,344,0.078488,1,344.0,169,49.13,7.001832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,Gaming Disorder and Effects of Gaming on Health.,\nPerhaps the virtual illusion has become toda...,virtual illusion today ’ “ reality ” inquisiti...,"[virtual, illusion, today, ’, “, reality, ”, i...",28,45,-0.232877,535,0.136449,1,535.0,210,39.25,13.630573
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,What is the repercussion of the environment du...,\nWhat is COVID 19 pandemic?\nOn 31st December...,covid 19 pandemic 31st 2019 coronavirus identi...,"[covid, 19, pandemic, 31st, 2019, coronavirus,...",6,24,-0.600000,335,0.089552,1,335.0,128,38.21,8.767338
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,"\nEpidemics, in general, have both direct and ...",epidemics general direct indirect costs direct...,"[epidemics, general, direct, indirect, costs, ...",26,54,-0.350000,564,0.141844,1,564.0,276,48.94,11.524315
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,\nCOVID 19 has bought the world to its knees. ...,covid 19 bought world knees businesses shut tr...,"[covid, 19, bought, world, knees, businesses, ...",22,35,-0.228070,430,0.132558,1,430.0,178,41.40,10.386473


In [159]:
df.to_csv("black-coffer-data")